In [64]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.anova import anova_lm
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [65]:
# Chargement des données
file_path = 'data/immodataParis.csv'
data = pd.read_csv(file_path)

data = data[['Arrondissement', 'Prix (€)',
       'Prix mensuel (€)', 'Pièce(s)', 'Surface (m2)', 'Date de vente']].copy()

df = pd.DataFrame(data)
df.head(15)

,Arrondissement,Prix (€),Prix mensuel (€),Pièce(s),Surface (m2),Date de vente
0,75001,750400,14431,2,52,22/05/2023
1,75001,330000,14348,1,23,28/04/2023
2,75001,360100,15657,1,23,29/03/2023
3,75001,286123,11005,1,26,10/02/2023
4,75001,411636,12864,2,32,28/12/2022
5,75001,334120,12375,1,27,16/12/2022
6,75001,1920000,12800,5,150,09/11/2022
7,75001,406000,11278,2,36,13/10/2022
8,75001,280000,12727,2,22,05/10/2022
9,75001,50000,2500,1,20,19/09/2022


### Vérification de la multicolinéarité

In [66]:
# Définir les variables indépendantes (X) et la variable dépendante (y)
X = df[['Surface (m2)', 'Pièce(s)', 'Arrondissement']]
y = df['Prix (€)']
# Ajouter une constante pour l'interception
X = sm.add_constant(X)

vif = pd.DataFrame()
vif["Variables"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)

        Variables           VIF
0           const  1.702824e+08
1    Surface (m2)  1.039003e+00
2        Pièce(s)  1.033087e+00
3  Arrondissement  1.007166e+00


Si les valeurs du VIF sont supérieur à 5-10, il y a une multicollinéarité, comme on peut le constater ils sont tous autour de 1 donc pas de problème de multicollinéarité

In [67]:

# Ajuster le modèle de régression linéaire en utilisant une formule
formula = 'Q("Prix (€)") ~ Q("Surface (m2)") + Q("Pièce(s)")'
model = smf.ols(formula=formula, data=df).fit(cov_type='HC3')

# Résumé du modèle
print(model.summary())

# Multicolinéarité entre la surface et l'arrondissement

                            OLS Regression Results                            
Dep. Variable:          Q("Prix (€)")   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.661
Method:                 Least Squares   F-statistic:                     52.03
Date:                Wed, 22 May 2024   Prob (F-statistic):           1.22e-22
Time:                        22:03:09   Log-Likelihood:                -24136.
No. Observations:                1654   AIC:                         4.828e+04
Df Residuals:                    1651   BIC:                         4.829e+04
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           -1.8e+05   7.97e+0

In [68]:
# Effectuer l'ANOVA
anova_results = anova_lm(model)
print(anova_results)

                       df        sum_sq       mean_sq            F    PR(>F)
Q("Surface (m2)")     1.0  8.948917e+14  8.948917e+14  3226.239009  0.000000
Q("Pièce(s)")         1.0  4.233223e+11  4.233223e+11     1.526150  0.216867
Residual           1651.0  4.579531e+14  2.773792e+11          NaN       NaN


Pour la surface, la p-value associée est pratiquement nulle. Cela indique que la surface en mètres carrés est un prédicteur hautement significatif du prix des appartements, comme nous l'avons analysé plus tôt dans l'EDA.

Tandis que pour le nombre de pièce, la p-value associée étant bien supérieure à 0.05, cela indique que le nombre de pièces n'est pas un prédicteur significatif du prix des appartements dans ce modèle.

### Prédiction du prix

In [85]:
# Diviser les données en ensembles d'entraînement et de test
train_data, test_data = train_test_split(df, test_size=0.3, random_state=123)

# Ajuster le modèle de régression linéaire en utilisant une formule
formula = 'Q("Prix (€)") ~ Q("Pièce(s)")'
model = smf.ols(formula=formula, data=train_data).fit(cov_type='HC3')

# Résumé du modèle
print(model.summary())

# Prédire les prix sur les données de test
predictions = model.predict(test_data)

# Évaluer les performances du modèle
rmse = np.sqrt(mean_squared_error(test_data['Prix (€)'], predictions))
mae = mean_absolute_error(test_data['Prix (€)'], predictions)
mape = mean_absolute_percentage_error(test_data['Prix (€)'], predictions)
r2 = r2_score(test_data['Prix (€)'], predictions)

                            OLS Regression Results                            
Dep. Variable:          Q("Prix (€)")   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     1.079
Date:                Wed, 22 May 2024   Prob (F-statistic):              0.299
Time:                        22:13:35   Log-Likelihood:                -17602.
No. Observations:                1157   AIC:                         3.521e+04
Df Residuals:                    1155   BIC:                         3.522e+04
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      6.887e+05   3.83e+04     17.992

### Significativité du Modèle
Le coefficient de détermination (R²) est de 0.651, ce qui signifie que 65.1% de la variance du prix des appartements est expliquée par les variables incluses dans le modèle. Le R² ajusté, qui prend en compte le nombre de prédicteurs, est également de 0.651, suggérant que le modèle a une bonne capacité explicative avec un ajustement minimal pour le nombre de variables. La statistique F est de 35.79, avec une p-value associée très faible (8.35e-16), confirmant que le modèle global est statistiquement significatif.

### Interprétation des Coefficients
L'intercept du modèle est de -191100 €, avec une p-value de 0.056, ce qui est marginalement non significatif. Cela signifie qu'un appartement avec une surface de 0 m² et 0 pièces aurait un prix de -191100 €, une extrapolation non réaliste mais nécessaire pour la fonction linéaire. Le coefficient de la surface (m²) est de 15620 €, avec une p-value de 0.000, indiquant qu'une augmentation de 1 m² de la surface est associée à une augmentation du prix de 15620 €, toutes choses égales par ailleurs. Ce coefficient est hautement significatif. En revanche, le coefficient du nombre de pièces est de 1231.6114 €, avec une p-value de 0.567, ce qui n'est pas statistiquement significatif. Cela suggère que, dans le contexte de ce modèle, le nombre de pièces n'a pas un impact significatif sur le prix des appartements.

### Diagnostics du Modèle
Les diagnostics du modèle révèlent quelques préoccupations. La statistique Omnibus et sa p-value associée indiquent que les résidus ne sont pas normalement distribués. La statistique de Durbin-Watson est de 1.996, suggérant une faible autocorrélation des résidus. La statistique Jarque-Bera est extrêmement élevée, ce qui indique que les résidus sont loin d'une distribution normale. De plus, la kurtosis de 199.092 suggère une distribution des résidus avec des queues très lourdes, indiquant la présence de nombreux outliers. Enfin, le numéro de condition est de 118, ce qui est modérément élevé et peut indiquer une certaine multicolinéarité, bien que les VIFs précédemment calculés n'aient pas suggéré de problème majeur.



In [81]:
print(f'Mean Squared Error: {rmse}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Absolute Percentage Error: {mape}% accuracy')
print(f'R²: {r2}')

Mean Squared Error: 353200.01873946877
Mean Absolute Error: 197447.3273172899
Mean Absolute Percentage Error: 16.548022341202955% accuracy
R²: 0.7121100787043778


16.548022341202955